In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df=pd.read_csv("train.csv")
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
# text pre-processing

In [ ]:
df.drop(columns=["id"], axis=1,inplace=True)

In [ ]:
df.head()

,keyword,location,text,target
0,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   keyword   7552 non-null   object
 1   location  5080 non-null   object
 2   text      7613 non-null   object
 3   target    7613 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 238.0+ KB


In [ ]:
df["target"].value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [ ]:
#check for null values
blanks=[]
for indx,kd,lc,txt,tg in df.itertuples():
  if type(txt)==str:
    if txt.isspace():
      blanks.append(indx)
print(blanks)

[]


In [ ]:
#there are no missing or empty string in our dataset

In [ ]:
#text pre-processing

In [ ]:
import nltk 

from nltk.tokenize import word_tokenize
nltk.download("punkt")


from  nltk.corpus import stopwords
nltk.download("stopwords")

from nltk.stem import PorterStemmer, WordNetLemmatizer
nltk.download("wordnet")

nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
def clean_sent(text):
  #tokenization and case conversion
  token=word_tokenize(text.lower())

  #token--->list of tokens
  #removing non alpha char
  ftoken=[i for i in token if i.isalpha()]

  #ftoken-----> list
  sw=stopwords.words("english")
  stokens=[i for i in ftoken if i not in sw]

  #stokens--->list
  #lemmatization
  lemma=WordNetLemmatizer()
  ltoken=[lemma.lemmatize(i) for i in stokens]

  #ltoken--->list

  #joining all tokens
  return " ".join(ltoken)

In [ ]:
df["clean_txt"]=df["text"].apply(clean_sent)

In [ ]:
df.head()

,keyword,location,text,target,clean_txt
0,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,deed reason earthquake may allah forgive u
1,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,forest fire near la ronge sask canada
2,NaN,NaN,All residents asked to 'shelter in place' are ...,1,resident asked place notified officer evacuati...
3,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,people receive wildfire evacuation order calif...
4,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,got sent photo ruby alaska smoke wildfire pour...


In [ ]:
#bulind ML model

In [ ]:
x=df["clean_txt"]
y=df["target"]

In [ ]:
x

0              deed reason earthquake may allah forgive u
1                   forest fire near la ronge sask canada
2       resident asked place notified officer evacuati...
3       people receive wildfire evacuation order calif...
4       got sent photo ruby alaska smoke wildfire pour...
                              ...                        
7608    two giant crane holding bridge collapse nearby...
7609    thetawniest control wild fire california even ...
7610                              utc volcano hawaii http
7611    police investigating collided car little portu...
7612    latest home razed northern california wildfire...
Name: clean_txt, Length: 7613, dtype: object

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
x = cv.fit_transform(x).toarray()

In [ ]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(x,y, test_size=0.3, random_state=1)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB

from sklearn.metrics import classification_report

In [ ]:
def mymodel(model):
  model.fit(xtrain, ytrain)
  ypred = model.predict(xtest)
  print(classification_report(ytest, ypred))
  return model

In [ ]:
logreg = mymodel(LogisticRegression())

              precision    recall  f1-score   support

           0       0.80      0.87      0.83      1326
           1       0.79      0.69      0.74       958

    accuracy                           0.80      2284
   macro avg       0.80      0.78      0.79      2284
weighted avg       0.80      0.80      0.79      2284



In [ ]:
svm = mymodel(LinearSVC())

              precision    recall  f1-score   support

           0       0.78      0.84      0.81      1326
           1       0.75      0.67      0.71       958

    accuracy                           0.77      2284
   macro avg       0.77      0.76      0.76      2284
weighted avg       0.77      0.77      0.77      2284



In [ ]:
nb = mymodel(GaussianNB())

              precision    recall  f1-score   support

           0       0.76      0.48      0.59      1326
           1       0.52      0.79      0.63       958

    accuracy                           0.61      2284
   macro avg       0.64      0.64      0.61      2284
weighted avg       0.66      0.61      0.61      2284



In [ ]:
def checksentiment(text):
  text = clean_sent(text)
  newx = cv.transform([text]).toarray()
  pred = logreg.predict(newx)[0]
  return pred

In [ ]:
sent="there is a forest fire at spot pond, geese are fleeing across the street, I cannot save them all"
checksentiment(sent)

1

In [ ]:
sent1="Hey! How are you?"
checksentiment(sent1)

0